In [50]:
from pymongo import MongoClient
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

In [51]:
# MongoDB connection details
mongo_uri = "mongodb://localhost:27017/"
database_name = "research"
perfromance_issues_collection_name = "performance-issues"
performance_refactorings_collection_name = "performance-refactorings"
non_performance_refactorings_collection_name = "non-performance-refactorings"

# Connect to MongoDB
client = MongoClient(mongo_uri)
db = client[database_name]
performance_issues_collection = db[perfromance_issues_collection_name]
performance_refactorings_collection = db[performance_refactorings_collection_name]
non_performance_refactorings_collection = db[non_performance_refactorings_collection_name]

In [52]:
# Fetch performance refactorings
performance_refactorings = list(
    performance_refactorings_collection.find(
        {}, {"repo_fullname": 1, "commit_id": 1, "issue_number": 1, "pr_number": 1}
    )
)

# Extract relevant details from performance refactorings
performance_refactorings_commit_ids = {ref["commit_id"] for ref in performance_refactorings}


# Print the total number of performance refactoring commit IDs
print(
    f"Total number of performance refactoring commit IDs: {len(performance_refactorings_commit_ids)}"
)

# Fetch issues related to performance refactorings by matching repository full names, issue numbers, and PR numbers
query = {
    "commit_ids": {"$in": list(performance_refactorings_commit_ids)},
}
issues_related_to_performance_refactorings = list(
    performance_issues_collection.find(query)
)

# Print the total number of issues related to performance refactorings
print(
    f"Total number of issues related to performance refactorings: {len(issues_related_to_performance_refactorings)}"
)

# Extract commit IDs for each issue related to performance refactorings
commit_ids = []
for issue in issues_related_to_performance_refactorings:
    issue_commit_ids = issue.get("commit_ids", [])
    commit_ids.extend(issue_commit_ids)

commit_ids_set = set(commit_ids)

# Print the total number of commit IDs related to performance refactorings
print(
    f"Total number of commit IDs related to performance refactorings: {len(commit_ids_set)}"
)

# Count commit IDs found and not found in performance refactorings
in_perf_ref = len(commit_ids_set & performance_refactorings_commit_ids)
not_in_perf_ref = len(commit_ids_set - performance_refactorings_commit_ids)

print(f"Number of commit IDs with performance refactorings: {in_perf_ref}")
print(f"Number of commit IDs without performance refactorings: {not_in_perf_ref}")

# Additional debugging and validation steps

# Check for issues with empty or null commit_ids
empty_commit_ids_issues = [
    issue
    for issue in issues_related_to_performance_refactorings
    if not issue.get("commit_ids")
]
print(f"Number of issues with empty or null commit_ids: {len(empty_commit_ids_issues)}")



Total number of performance refactoring commit IDs: 2316
Total number of issues related to performance refactorings: 940
Total number of commit IDs related to performance refactorings: 5704
Number of commit IDs with performance refactorings: 2316
Number of commit IDs without performance refactorings: 3388
Number of issues with empty or null commit_ids: 0


In [55]:
# Query for documents where 'type' is null
non_performance_refactorings_without_refactorings = list(
    non_performance_refactorings_collection.find(
        {"type": None}, {"type": 1, "commit_id": 1}
    )
)

# Query for documents where 'type' is not null
non_performance_refactorings_with_refactorings = list(
    non_performance_refactorings_collection.find(
        {"type": {"$ne": None}}, {"type": 1, "commit_id": 1}
    )
)

# Convert both results to DataFrames
without_refactorings_df = pd.DataFrame(non_performance_refactorings_without_refactorings)
with_refactorings_df = pd.DataFrame(non_performance_refactorings_with_refactorings)

# Get the number of unique commit_ids in each group
commit_without_refactorings = without_refactorings_df["commit_id"].nunique()
commit_with_refactorings = with_refactorings_df["commit_id"].nunique()

# Print the results
print(
    f"Number of commits without refactorings: {commit_without_refactorings}"
)
print(
    f"Number of commits with refactorings: {commit_with_refactorings}"
)

Number of commits without refactorings: 1586697
Number of commits with refactorings: 612796


In [ ]:
data = {
    "Performance_related": [
        in_perf_ref,
        not_in_perf_ref,
    ],  # [Refactoring Present, Refactoring Absent]
    "Non_performance_related": [
        commit_with_refactorings,
        commit_without_refactorings,
    ],  # [Refactoring Present, Refactoring Absent]
}

# Create a DataFrame
df = pd.DataFrame(data, index=["Refactoring_Present", "Refactoring_Absent"])
df["Total"] = df.sum(axis=1)
df.loc["Total"] = df.sum()
df

print(df)

chi2, p, dof, expected = chi2_contingency(df.iloc[0:2, 0:2])

# Calculate the odds ratio
odds_ratio = (in_perf_ref / not_in_perf_ref) / (
    commit_with_refactorings / commit_without_refactorings
)

log_p_value = (
    -np.log10(p) 
)  # Check for zero p-value


chi2, p, dof, expected, odds_ratio, log_p_value

                     Performance_related  Non_performance_related    Total
Refactoring_Present                 2316                   612796   615112
Refactoring_Absent                  3388                  1586697  1590085
Total                               5704                  2199493  2205197


(458.63945163167995,
 9.505413872093378e-102,
 1,
 array([[   1591.05914256,  613520.94085744],
        [   4112.94085744, 1585972.05914256]]),
 1.7699998608194958,
 101.02202896891393)